In [12]:
import pandas as pd

# Load predictions from csvs
uk_df_dct = {}
uk_df_dct['actuals_df'] = pd.read_excel('../processed_data/election_forecast_tracker.xlsx', sheet_name='actuals', index_col=0)
uk_df_dct['polls_avg_df'] = pd.read_excel('../processed_data/election_forecast_tracker.xlsx', sheet_name='polls_average', index_col=0)
uk_df_dct['polls_model_df'] = pd.read_excel('../processed_data/election_forecast_tracker.xlsx', sheet_name='polls_model', index_col=0)
uk_df_dct['polls_eco_df'] = pd.read_excel('../processed_data/election_forecast_tracker.xlsx', sheet_name='polls_eco', index_col=0)
uk_df_dct['polls_alt_df'] = pd.read_excel('../processed_data/election_forecast_tracker.xlsx', sheet_name='polls_alt', index_col=0)
uk_df_dct['polls_eco_alt_df'] = pd.read_excel('../processed_data/election_forecast_tracker.xlsx', sheet_name='polls_eco_alt', index_col=0)

# Load Scottish forecast data
scotland_df = pd.read_excel('../processed_data/election_forecast_tracker.xlsx', sheet_name='scotland', index_col=0)

# Loading hex constituency mapping
hex_map = pd.read_excel('../processed_data/election_forecast_tracker.xlsx', sheet_name='hex_codes', index_col=0)
constituency_map = pd.read_excel('../processed_data/election_forecast_tracker.xlsx', sheet_name='constituency_coords', index_col=0)

# Define the forecast models
forecast_models = uk_df_dct.copy()

# Import the test bias data
test_bias = pd.read_csv('../processed_data/last_bias.csv', index_col=0)
def calculate_predicted_vote_share(forecast, bias_scores):
    predicted_vote_share = {}
    for party, forecast_share in forecast.items():
        bias_score = bias_scores.get(f"{party}_Bias_Score", 0)
        predicted_vote_share[party] = forecast_share + bias_score
    return predicted_vote_share
def determine_winner(predicted_vote_share):
    return max(predicted_vote_share, key=predicted_vote_share.get)
def determine_winners(forecast, bias_dataframe):
    winners = []
    for index, row in bias_dataframe.iterrows():
        predicted_vote_share = calculate_predicted_vote_share(forecast, row)
        winner = determine_winner(predicted_vote_share)
        winners.append({'Constituency': index, 'Winner': winner})
    return pd.DataFrame(winners)

# Separate Scottish and non-Scottish constituencies in the bias data
scottish_bias = test_bias[test_bias['Scotland'] == True]
uk_test_bias = test_bias[test_bias['Scotland'] == False]

# Container for results
total_constituencies_results = {}
constituency_winners_results = {}

# Loop through each model and each forecast year
for model_name, forecast_df in forecast_models.items():
    model_results = {}
    constituency_winners = {}
    for year in forecast_df.columns:
        forecast = forecast_df[year].dropna().to_dict()
        scotland_forecast = scotland_df[year].dropna().to_dict()

        # Determine winners for UK (excluding Scotland)
        uk_winners_df = determine_winners(forecast, uk_test_bias)
        uk_total_constituencies = uk_winners_df['Winner'].value_counts().reset_index()
        uk_total_constituencies.columns = ['Party', 'Total_Constituencies']

        # Determine winners for Scotland
        scotland_winners_df = determine_winners(scotland_forecast, scottish_bias)
        scotland_total_constituencies = scotland_winners_df['Winner'].value_counts().reset_index()
        scotland_total_constituencies.columns = ['Party', 'Total_Constituencies']

        # Concatenate the results
        total_constituencies = pd.concat([uk_total_constituencies, scotland_total_constituencies]).groupby('Party').sum().reset_index()

        # Add the original vote share to the table
        total_constituencies['Vote_Share'] = total_constituencies['Party'].map(forecast_df[year])

        # Concatenate the constituency winners
        winners_df = pd.concat([uk_winners_df, scotland_winners_df])
        winners_df = pd.merge(winners_df,hex_map,how='left',left_on='Winner',right_on='party_code')
        winners_df = pd.merge(winners_df,constituency_map,how='left',left_on='Constituency',right_on='constituency_name')
        model_results[year] = total_constituencies
        constituency_winners[year] = winners_df
    total_constituencies_results[model_name] = model_results
    constituency_winners_results[model_name] = constituency_winners


In [13]:
winners_df

,Constituency,Winner,elected_mp_party,elected_mp_party_name,color,constituency_id,coord_one,coord_two
0,Aberafan Maesteg,LAB,Lab,Labour,#dd0018,NaN,NaN,NaN
1,Aldershot,CON,Con,Conservative,#005af0,E14000530,-3.0,-11.0
2,Aldridge-Brownhills,CON,Con,Conservative,#005af0,E14000531,-3.0,-1.0
3,Altrincham and Sale West,LAB,Lab,Labour,#dd0018,E14000532,-7.0,3.0
4,Alyn and Deeside,LAB,Lab,Labour,#dd0018,W07000043,-10.0,1.0
...,...,...,...,...,...,...,...,...
645,Perth and Kinross-shire,SNP,SNP,SNP,#fff293,NaN,NaN,NaN
646,Rutherglen,LAB,Lab,Labour,#dd0018,NaN,NaN,NaN
647,Stirling and Strathallan,CON,Con,Conservative,#005af0,NaN,NaN,NaN
648,West Aberdeenshire and Kincardine,LAB,Lab,Labour,#dd0018,S14000058,-4.0,24.0


In [14]:

# Create an Excel writer
with pd.ExcelWriter('election_results.xlsx') as writer:

    # Write total constituencies results to Excel
    for model_name, results in total_constituencies_results.items():
        for year, df in results.items():
            sheet_name = f"{model_name}_{year}_total"
            df.to_excel(writer, sheet_name=sheet_name, index=False)

    # Write constituency winners results to Excel
    for model_name, results in constituency_winners_results.items():
        for year, df in results.items():
            sheet_name = f"{model_name}_{year}_winners"
            df.to_excel(writer, sheet_name=sheet_name, index=False)

# Display the results for verification
for model_name, results in total_constituencies_results.items():
    print(f"Results for model: {model_name}")
    for year, df in results.items():
        print(f"Year: {year}")
        print(df)
        print()

for model_name, results in constituency_winners_results.items():
    print(f"Constituency Winners for model: {model_name}")
    for year, df in results.items():
        print(f"Year: {year}")
        print(df)
        print()

Results for model: actuals_df
Year: 2010
  Party  Total_Constituencies  Vote_Share
0   CON                   321        36.2
1   GRE                     1         0.7
2   LAB                   264        29.2
3   LIB                    40        23.5
4   OTH                    19        10.8
5   PLC                     3         0.6
6   SNP                     2         3.4

Year: 2015
  Party  Total_Constituencies  Vote_Share
0   CON                   337        36.9
1   GRE                     1         3.8
2   LAB                   227        30.4
3   LIB                     8         7.9
4   OTH                    19         3.1
5   PLC                     4         0.6
6   SNP                    54         4.7

Year: 2017
  Party  Total_Constituencies  Vote_Share
0   CON                   326        42.4
1   GRE                     1         1.6
2   LAB                   246        40.0
3   LIB                     7         7.4
4   OTH                    19         3.3
5   PLC    